<a href="https://colab.research.google.com/github/AroubAlRizq/advanced_ai_exercises/blob/main/Lab1Solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch, torchvision, torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.utils import make_grid
import math

device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
batch_size=128; z_dim=64; g_lr=d_lr=2e-4; num_epochs=3
transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_ds=torchvision.datasets.MNIST('./data',True,download=True,transform=transform)
train_loader=DataLoader(train_ds,batch_size=batch_size,shuffle=True,num_workers=2,pin_memory=True)

100%|██████████| 9.91M/9.91M [00:00<00:00, 39.3MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 1.10MB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 9.80MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 6.30MB/s]


In [4]:
class Generator(nn.Module):
    def __init__(self,z_dim=64):
        super().__init__()
        self.net=nn.Sequential(
            nn.ConvTranspose2d(z_dim,128,7,1,0),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            nn.ConvTranspose2d(128,64,4,2,1),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            nn.ConvTranspose2d(64,1,4,2,1),
            nn.Tanh())
             # TODO: implement upsampling to 28x28 + Tanh
    def forward(self,z):
      z=z.view(z.size(0),z_dim,1,1);
      return self.net(z)  # BUG reshape

class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.net=nn.Sequential(nn.Conv2d(1,32,4,2,1),
                               nn.LeakyReLU(0.2,True),
                               nn.Conv2d(32,64,4,2,1),
                               nn.BatchNorm2d(64),
                               nn.LeakyReLU(0.2,True),
                               nn.Conv2d(64,1,7,1,0),
                               )
    def forward(self,x):
      return self.net(x).view(x.size(0),1)


In [5]:
criterion=nn.BCEWithLogitsLoss()
G=Generator(z_dim).to(device);
D=Discriminator().to(device)
opt_g=torch.optim.Adam(G.parameters(),lr=g_lr,betas=(0.5,0.999))
opt_d=torch.optim.Adam(D.parameters(),lr=d_lr,betas=(0.5,0.999))

In [6]:
@torch.no_grad()
def sample_grid(G,n=16):
    z=torch.randn(n,64,device=device).view(n,64,1,1)
    fake=G(z).cpu()
    return make_grid(fake,nrow=int(math.sqrt(n)),normalize=True,value_range=(-1,1))

In [7]:
def train_discriminator_step(real):
    real = real.to(device); b=real.size(0)
    z=torch.randn(b,z_dim,device=device);
    fake=G(z.view(b,z_dim,1,1))
    loss_d=criterion(D(real),
                torch.ones(b,1,device=device)) # D(x) should be 1
    +criterion(D(fake),torch.zeros(b,1,device=device))  # D(G(x)) should be 0
    loss_d.backward();
    opt_d.step()  # BUG
    # TODO: Goodfellow D loss with detach on fake

def train_generator_step(b):
    z=torch.randn(b,z_dim,device=device);
    fake=G(z.view(b,z_dim,1,1))
    loss_g=criterion(D(fake),
                torch.ones(b,1,device=device))  # BUG non-sat
    loss_g.backward()  # BUG missing step
    opt_d.step()
    # TODO: non-saturating G loss


In [ ]:
from tqdm import tqdm
for epoch in range(num_epochs):
    pbar=tqdm(train_loader)
    for real,_ in pbar:
        real=real.to(device)
        d=train_discriminator_step(real)
        g=train_generator_step(real.size(0))
        pbar.set_postfix(d_loss=d,g_loss=g)
print('Done')


  0%|          | 0/469 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
 44%|████▎     | 205/469 [01:46<02:14,  1.96it/s, d_loss=None, g_loss=None]